# HW1
## 105072123 黃海茵

In [5]:
import librosa
import pretty_midi
import mir_eval
import scipy
import os
import math

In [6]:
y_key = {
    0: [0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1],
    1: [1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0],
    2: [0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1],   
    3: [1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1],
    4: [1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0],
    5: [0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1],
    6: [1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0],
    7: [0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1],
    8: [1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0],
    9: [0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1],
    10: [1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1],
    11: [1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0],
    12: [1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1],
    13: [1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0],
    14: [0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1],
    15: [1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0],
    16: [0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1],
    17: [1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0],
    18: [0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1],
    19: [1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1],
    20: [1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0],
    21: [0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1],
    22: [1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0],
    23: [0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1],
}

In [7]:
index_key = {'A major': 0, 'Bb major': 1, 'B major': 2, 'C major': 3, 'Db major': 4, 'D major': 5,
             'Eb major': 6, 'E major': 7, 'F major': 8, 'Gb major': 9, 'G major': 10, 'Ab major': 11,
             'A minor': 12, 'Bb minor': 13, 'B minor': 14, 'C minor': 15, 'Db minor': 16, 'D minor': 17,
             'Eb minor': 18, 'E minor': 19, 'F minor': 20, 'Gb minor': 21, 'G minor': 22, 'Ab minor': 23}

In [8]:
index_key2 = {'A': 0, 'A+': 1, 'B-': 1, 'B': 2, 'C': 3, 'C+': 4, 'D-': 4, 'D': 5,
              'D+': 6, 'E-': 6, 'E': 7, 'F': 8, 'F+': 9, 'G-': 9, 'G': 10, 'G+': 11, 'A-': 11,
              'a': 12, 'a+': 13, 'b-': 13, 'b': 14, 'c': 15, 'c+':16, 'd-': 16, 'd': 17,
              'd+':18, 'e-': 18, 'e': 19, 'f': 20, 'f+': 21, 'g-': 21, 'g': 22, 'g+': 23, 'a-': 23}

In [9]:
list_key = ['A major', 'Bb major', 'B major', 'C major', 'Db major', 'D major',
            'Eb major', 'E major', 'F major', 'Gb major', 'G major', 'Ab major',
            'A minor', 'Bb minor', 'B minor', 'C minor', 'Db minor', 'D minor',
            'Eb minor', 'E minor', 'F minor', 'Gb minor', 'G minor', 'Ab minor']

In [10]:
KS_major = [6.35, 2.23, 3.48, 2.33, 4.38, 4.09, 2.52, 5.19, 2.39, 3.66, 2.29, 2.88]
KS_minor = [6.33, 2.68, 3.52, 5.38, 2.60, 3.53, 2.54, 4.75, 3.98, 2.69, 3.34, 3.17]
KS_key = {}

for i in range(12):
    KS_key[i] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    for j in range(12):
        pos = (9+i+j) % 12
        KS_key[i][pos] = KS_major[j]
        
for i in range(12, 24):
    KS_key[i] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    for j in range(12):
        pos = (9+i+j) % 12
        KS_key[i][pos] = KS_minor[j]

# Task1_Q1
## GiantSteps

In [59]:
music_giant = []
label_giant = []
pieces_all = 0
correct_all = 0

for file in os.listdir('giantsteps/audio/'):
    if file.endswith('.wav'):
        f = open(('giantsteps/annotations/key/' + file).replace('wav', 'key'), 'r')
        key = index_key[f.read()]
        
        y, sr = librosa.load('giantsteps/audio/' + file)
        chroma = librosa.feature.chroma_stft(y=y, sr=sr)
        x = []
        for z in chroma:
            x.append(sum(z))    
        j = (x.index(max(x))+3) % 12
        RM = scipy.stats.pearsonr(x, y_key[j])[0]
        Rm = scipy.stats.pearsonr(x, y_key[j+12])[0]
        if RM > Rm:
            music_key = j
        else:
            music_key = j + 12
        
        f = open(('giantsteps/annotations/key/' + file).replace('wav', 'key'), 'r')
        label_key = index_key[f.read()]
        f.close()

        pieces_all += 1
        if music_key == label_key:
            correct_all += 1
                
        music_giant.append(music_key)
        label_giant.append(label_key)

In [64]:
print('giantsteps ACC: ' + str(correct_all/pieces_all))

giantsteps ACC: 0.27879799666110183


## GTZAN

In [107]:
genre = ['blues', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']
music = {}
label = {}
pieces_all = 0
correct_all = 0

for g in genre:
    pieces_gen = 0
    correct_gen = 0
    music[g] = []
    label[g] = []
    for i in range(100):
        wav = 'GTZAN_wav/' + g + '/' + g + '.' + str(i).zfill(5) + '.wav'
        txt =  'gtzan_key/genres/' + g + '/' + g + '.' + str(i).zfill(5) + '.lerch.txt'

        y, sr = librosa.load(wav)
        chroma = librosa.feature.chroma_stft(y=y, sr=sr)
        x = []
        for z in chroma:
            x.append(sum(z))    
        j = (x.index(max(x))+3) % 12
        RM = scipy.stats.pearsonr(x, y_key[j])[0]
        Rm = scipy.stats.pearsonr(x, y_key[j+12])[0]
        if RM > Rm:
            music_key = j
        else:
            music_key = j + 12

        f = open(txt, 'r');
        label_key = int(f.read())
        f.close()

        if label_key >= 0:
            pieces_gen += 1
            if music_key == label_key:
                correct_gen += 1
                
        music[g].append(music_key)
        label[g].append(label_key)

    pieces_all += pieces_gen
    correct_all += correct_gen
    print('{:<15}'.format(g + ' ACC: ') + str(correct_gen/pieces_gen))

blues ACC:     0.07142857142857142
country ACC:   0.32323232323232326
disco ACC:     0.3163265306122449
hiphop ACC:    0.13580246913580246
jazz ACC:      0.16455696202531644
metal ACC:     0.24731182795698925
pop ACC:       0.4148936170212766
reggae ACC:    0.32989690721649484
rock ACC:      0.3469387755102041


In [108]:
print('{:<15}'.format('overall ACC: ') + str(correct_all/pieces_all)) 

overall ACC:   0.26523297491039427


# Task1_Q2
## GiantSteps

In [72]:
def task1_giant_q2(gamma):
    
    music_giant_q2 = []
    label_giant_q2 = []
    pieces_all = 0
    correct_all = 0
    
    for file in os.listdir('giantsteps/audio/'):
        if file.endswith('.wav'):
            f = open(('giantsteps/annotations/key/' + file).replace('wav', 'key'), 'r')
            key = index_key[f.read()]

            y, sr = librosa.load('giantsteps/audio/' + file)
            chroma = librosa.feature.chroma_stft(y=y, sr=sr)
            x = []
            for z in chroma:
                for i in range(len(z)):
                    z[i] = math.log(1 + gamma*abs(z[i]))
                x.append(sum(z))    
            j = (x.index(max(x))+3) % 12
            RM = scipy.stats.pearsonr(x, y_key[j])[0]
            Rm = scipy.stats.pearsonr(x, y_key[j+12])[0]
            if RM > Rm:
                music_key = j
            else:
                music_key = j + 12

            f = open(('giantsteps/annotations/key/' + file).replace('wav', 'key'), 'r')
            label_key = index_key[f.read()]
            f.close()

            pieces_all += 1
            if music_key == label_key:
                correct_all += 1

            music_giant_q2.append(music_key)
            label_giant_q2.append(label_key)
            
    return correct_all/pieces_all

In [75]:
for i in range(4):
    gamma = math.pow(10, i)
    ACC = task1_giant_q2(gamma)
    print('gamma = ' + str(gamma) + ', giantsteps ACC: ' + str(ACC))

gamma = 1.0, giantsteps ACC: 0.2671118530884808
gamma = 10.0, giantsteps ACC: 0.24207011686143573
gamma = 100.0, giantsteps ACC: 0.2370617696160267
gamma = 1000.0, giantsteps ACC: 0.2337228714524207


## GTZAN

In [109]:
def task1_gtzan_q2(gamma):

    music_q2 = {}
    label_q2 = {}
    pieces_all = 0
    correct_all = 0

    for g in genre:
        pieces_gen = 0
        correct_gen = 0
        music_q2[g] = []
        label_q2[g] = []
        for i in range(100):
            wav = 'GTZAN_wav/' + g + '/' + g + '.' + str(i).zfill(5) + '.wav'
            txt =  'gtzan_key/genres/' + g + '/' + g + '.' + str(i).zfill(5) + '.lerch.txt'

            y, sr = librosa.load(wav)
            chroma = librosa.feature.chroma_stft(y=y, sr=sr)
            x = []
            for z in chroma:
                for i in range(len(z)):
                    z[i] = math.log(1 + gamma*abs(z[i]))
                x.append(sum(z))    
            j = (x.index(max(x))+3) % 12
            RM = scipy.stats.pearsonr(x, y_key[j])[0]
            Rm = scipy.stats.pearsonr(x, y_key[j+12])[0]
            if RM > Rm:
                music_key = j
            else:
                music_key = j + 12

            f = open(txt, 'r');
            label_key = int(f.read())
            f.close()

            if label_key >= 0:
                pieces_gen += 1
                if music_key == label_key:
                    correct_gen += 1

            music_q2[g].append(music_key)
            label_q2[g].append(label_key)

        pieces_all += pieces_gen
        correct_all += correct_gen
        print('{:<30}'.format('gamma = ' + str(gamma) + ', ' + g + ' ACC: ') + str(correct_gen/pieces_gen))
        
    return correct_all/pieces_all

In [74]:
for i in range(4):
    gamma = math.pow(10, i)
    ACC = task1_gtzan_q2(gamma)
    print('{:<30}'.format('gamma = ' + str(gamma) + ', overall ACC: ') + str(ACC))

gamma = 1.0, blues ACC:       0.07142857142857142
gamma = 1.0, country ACC:     0.3434343434343434
gamma = 1.0, disco ACC:       0.32653061224489793
gamma = 1.0, hiphop ACC:      0.13580246913580246
gamma = 1.0, jazz ACC:        0.16455696202531644
gamma = 1.0, metal ACC:       0.21505376344086022
gamma = 1.0, pop ACC:         0.40425531914893614
gamma = 1.0, reggae ACC:      0.31958762886597936
gamma = 1.0, rock ACC:        0.336734693877551
gamma = 1.0, overall ACC:     0.2616487455197133
gamma = 10.0, blues ACC:      0.04081632653061224
gamma = 10.0, country ACC:    0.31313131313131315
gamma = 10.0, disco ACC:      0.2857142857142857
gamma = 10.0, hiphop ACC:     0.14814814814814814
gamma = 10.0, jazz ACC:       0.11392405063291139
gamma = 10.0, metal ACC:      0.1935483870967742
gamma = 10.0, pop ACC:        0.39361702127659576
gamma = 10.0, reggae ACC:     0.28865979381443296
gamma = 10.0, rock ACC:       0.30612244897959184
gamma = 10.0, overall ACC:    0.23536439665471923
gamma 

# Task1_Q3
## GiantSteps

In [78]:
pieces_all = 0
correct_all = 0

for i in range(len(music_giant)):
    ref_key = list_key[music_giant[i]]
    est_key = list_key[label_giant[i]]
    correct_all += mir_eval.key.weighted_score(ref_key, est_key)
    pieces_all += 1

In [79]:
print('giantsteps new ACC: ' + str(correct_all/pieces_all))

giantsteps new ACC: 0.33856427378964904


## GTZAN

In [110]:
pieces_all = 0
correct_all = 0

for g in genre:
    pieces_gen = 0
    correct_gen = 0
    for i in range(100):
        if label[g][i] >= 0:
            ref_key = list_key[music[g][i]]
            est_key = list_key[label[g][i]]
            correct_gen += mir_eval.key.weighted_score(ref_key, est_key)
            pieces_gen += 1
        
    correct_all += correct_gen
    pieces_all += pieces_gen
    print('{:<20}'.format(g + ' new ACC: ') + str(correct_gen/pieces_gen))        

blues new ACC:      0.18571428571428564
country new ACC:    0.3363636363636363
disco new ACC:      0.3469387755102041
hiphop new ACC:     0.18395061728395057
jazz new ACC:       0.2240506329113924
metal new ACC:      0.3053763440860214
pop new ACC:        0.4542553191489362
reggae new ACC:     0.35463917525773203
rock new ACC:       0.41122448979591836


In [111]:
print('{:<20}'.format('overall new ACC: ') + str(correct_all/pieces_all))   

overall new ACC:    0.31529271206690557


# Task2_Q4-1

## GiantSteps

In [144]:
music_giant_ks = []
label_giant_ks = []
pieces_all = 0
correct_all = 0

for file in os.listdir('giantsteps/audio/'):
    if file.endswith('.wav'):
        f = open(('giantsteps/annotations/key/' + file).replace('wav', 'key'), 'r')
        key = index_key[f.read()]
        
        y, sr = librosa.load('giantsteps/audio/' + file)
        chroma = librosa.feature.chroma_stft(y=y, sr=sr)
        x = []
        R_max = -1
        for z in chroma:
            x.append(sum(z))
        for j in range(24):
            R = scipy.stats.pearsonr(x, KS_key[j])[0]
            if R > R_max:
                R_max = R
                music_key = j
        
        f = open(('giantsteps/annotations/key/' + file).replace('wav', 'key'), 'r')
        label_key = index_key[f.read()]
        f.close()

        pieces_all += 1
        if music_key == label_key:
            correct_all += 1
                
        music_giant_ks.append(music_key)
        label_giant_ks.append(label_key)

In [145]:
print('giantsteps K-S ACC: ' + str(correct_all/pieces_all))

giantsteps K-S ACC: 0.4257095158597663


## GTZAN

In [146]:
music_ks = {}
label_ks = {}
pieces_all = 0
correct_all = 0

for g in genre:
    pieces_gen = 0
    correct_gen = 0
    music_ks[g] = []
    label_ks[g] = []
    for i in range(100):
        wav = 'GTZAN_wav/' + g + '/' + g + '.' + str(i).zfill(5) + '.wav'
        txt =  'gtzan_key/genres/' + g + '/' + g + '.' + str(i).zfill(5) + '.lerch.txt'

        y, sr = librosa.load(wav)
        chroma = librosa.feature.chroma_stft(y=y, sr=sr)
        x = []
        R_max = -1
        for z in chroma:
            x.append(sum(z))
        for j in range(24):
            R = scipy.stats.pearsonr(x, KS_key[j])[0]
            if R > R_max:
                R_max = R
                music_key = j

        f = open(txt, 'r');
        label_key = int(f.read())
        f.close()

        if label_key >= 0:
            pieces_gen += 1
            if music_key == label_key:
                correct_gen += 1
                
        music_ks[g].append(music_key)
        label_ks[g].append(label_key)

    pieces_all += pieces_gen
    correct_all += correct_gen
    print('{:<20}'.format(g + ' K-S ACC: ') + str(correct_gen/pieces_gen))

blues K-S ACC:      0.19387755102040816
country K-S ACC:    0.494949494949495
disco K-S ACC:      0.3469387755102041
hiphop K-S ACC:     0.18518518518518517
jazz K-S ACC:       0.2911392405063291
metal K-S ACC:      0.34408602150537637
pop K-S ACC:        0.5851063829787234
reggae K-S ACC:     0.5773195876288659
rock K-S ACC:       0.42857142857142855


In [147]:
print('{:<20}'.format('overall K-S ACC: ') + str(correct_all/pieces_all))

overall K-S ACC:    0.38829151732377537


# Task2_Q4-2

In [155]:
def task2_giant_q2(gamma):
    
    music_giant2_q2 = []
    label_giant2_q2 = []
    pieces_all = 0
    correct_all = 0
    
    for file in os.listdir('giantsteps/audio/'):
        if file.endswith('.wav'):
            f = open(('giantsteps/annotations/key/' + file).replace('wav', 'key'), 'r')
            key = index_key[f.read()]

            y, sr = librosa.load('giantsteps/audio/' + file)
            chroma = librosa.feature.chroma_stft(y=y, sr=sr)
            x = []
            R_max = -1
            for z in chroma:
                for i in range(len(z)):
                    z[i] = math.log(1 + gamma*abs(z[i]))
                x.append(sum(z))
                
            for j in range(24):
                R = scipy.stats.pearsonr(x, KS_key[j])[0]
                if R > R_max:
                    R_max = R
                    music_key = j

            f = open(('giantsteps/annotations/key/' + file).replace('wav', 'key'), 'r')
            label_key = index_key[f.read()]
            f.close()

            pieces_all += 1
            if music_key == label_key:
                correct_all += 1

            music_giant2_q2.append(music_key)
            label_giant2_q2.append(label_key)
            
    return correct_all/pieces_all

In [158]:
for i in range(4):
    gamma = math.pow(10, i)
    ACC = task2_giant_q2(gamma)
    print('gamma = ' + str(gamma) + ', giantsteps K-S ACC: ' + str(ACC))

gamma = 1.0, giantsteps K-S ACC: 0.4240400667779633
gamma = 10.0, giantsteps K-S ACC: 0.3973288814691152
gamma = 100.0, giantsteps K-S ACC: 0.3772954924874791
gamma = 1000.0, giantsteps K-S ACC: 0.3789649415692821


In [156]:
def task2_gtzan_q2(gamma):

    music2_q2 = {}
    label2_q2 = {}
    pieces_all = 0
    correct_all = 0

    for g in genre:
        pieces_gen = 0
        correct_gen = 0
        music2_q2[g] = []
        label2_q2[g] = []
        for i in range(100):
            wav = 'GTZAN_wav/' + g + '/' + g + '.' + str(i).zfill(5) + '.wav'
            txt =  'gtzan_key/genres/' + g + '/' + g + '.' + str(i).zfill(5) + '.lerch.txt'

            y, sr = librosa.load(wav)
            chroma = librosa.feature.chroma_stft(y=y, sr=sr)
            x = []
            R_max = -1
            for z in chroma:
                for i in range(len(z)):
                    z[i] = math.log(1 + gamma*abs(z[i]))
                x.append(sum(z))
                
            for j in range(24):
                R = scipy.stats.pearsonr(x, KS_key[j])[0]
                if R > R_max:
                    R_max = R
                    music_key = j

            f = open(txt, 'r');
            label_key = int(f.read())
            f.close()

            if label_key >= 0:
                pieces_gen += 1
                if music_key == label_key:
                    correct_gen += 1

            music2_q2[g].append(music_key)
            label2_q2[g].append(label_key)

        pieces_all += pieces_gen
        correct_all += correct_gen
        print('{:<35}'.format('gamma = ' + str(gamma) + ', ' + g + ' K-S ACC: ') + str(correct_gen/pieces_gen))
        
    return correct_all/pieces_all

In [157]:
for i in range(4):
    gamma = math.pow(10, i)
    ACC = task2_gtzan_q2(gamma)
    print('{:<35}'.format('gamma = ' + str(gamma) + ', overall K-S ACC: ') + str(ACC))

gamma = 1.0, blues K-S ACC:        0.1836734693877551
gamma = 1.0, country K-S ACC:      0.48484848484848486
gamma = 1.0, disco K-S ACC:        0.37755102040816324
gamma = 1.0, hiphop K-S ACC:       0.1728395061728395
gamma = 1.0, jazz K-S ACC:         0.26582278481012656
gamma = 1.0, metal K-S ACC:        0.3333333333333333
gamma = 1.0, pop K-S ACC:          0.5531914893617021
gamma = 1.0, reggae K-S ACC:       0.5567010309278351
gamma = 1.0, rock K-S ACC:         0.41836734693877553
gamma = 1.0, overall K-S ACC:      0.37753882915173237
gamma = 10.0, blues K-S ACC:       0.19387755102040816
gamma = 10.0, country K-S ACC:     0.4444444444444444
gamma = 10.0, disco K-S ACC:       0.35714285714285715
gamma = 10.0, hiphop K-S ACC:      0.1728395061728395
gamma = 10.0, jazz K-S ACC:        0.24050632911392406
gamma = 10.0, metal K-S ACC:       0.27956989247311825
gamma = 10.0, pop K-S ACC:         0.5319148936170213
gamma = 10.0, reggae K-S ACC:      0.4948453608247423
gamma = 10.0, rock 

# Task2_Q4-3
## GiantSteps

In [148]:
pieces_all = 0
correct_all = 0

for i in range(len(music_giant_ks)):
    ref_key = list_key[music_giant_ks[i]]
    est_key = list_key[label_giant_ks[i]]
    correct_all += mir_eval.key.weighted_score(ref_key, est_key)
    pieces_all += 1

In [149]:
print('giantsteps new K-S ACC: ' + str(correct_all/pieces_all))

giantsteps new K-S ACC: 0.4662771285475789


## GTZAN

In [150]:
pieces_all = 0
correct_all = 0

for g in genre:
    pieces_gen = 0
    correct_gen = 0
    for i in range(100):
        if label_ks[g][i] >= 0:
            ref_key = list_key[music_ks[g][i]]
            est_key = list_key[label_ks[g][i]]
            correct_gen += mir_eval.key.weighted_score(ref_key, est_key)
            pieces_gen += 1
        
    correct_all += correct_gen
    pieces_all += pieces_gen
    print('{:<25}'.format(g + ' new K-S ACC: ') + str(correct_gen/pieces_gen))        

blues new K-S ACC:       0.29489795918367323
country new K-S ACC:     0.5141414141414141
disco new K-S ACC:       0.4204081632653063
hiphop new K-S ACC:      0.24444444444444446
jazz new K-S ACC:        0.3556962025316456
metal new K-S ACC:       0.41397849462365593
pop new K-S ACC:         0.6202127659574468
reggae new K-S ACC:      0.5907216494845361
rock new K-S ACC:        0.4765306122448979


In [151]:
print('{:<25}'.format('overall new K-S ACC: ') + str(correct_all/pieces_all))

overall new K-S ACC:     0.4416965352449223


# Task3_Q5
## BPS-FH

In [46]:
pieces_all = 0
correct_all = 0
correct_new_all = 0

for file in os.listdir('BPS/audio/'):
    
    y, sr = librosa.load('BPS/audio/' + file)

    label_bps = []
    f = open('BPS/label/REF_key_' + file.replace('wav', 'txt'), 'r')
    for line in f.read().split('\n'):
        line = line.split('\t')
        if len(line) > 1:
            label_bps.append(index_key2[line[1]])
    f.close()

    pieces_gen = 0
    correct_gen = 0
    correct_new_gen = 0
    
    for i in range(math.floor(len(y)/sr)):
        if i < 15:
            start = 0
            end = (i+15) * sr
        elif i > math.floor(len(y)/sr)-15:
            start = (i-15) * sr
            end = math.floor(len(y)/sr) * sr
        else:
            start = (i-15) * sr
            end = (i+15) * sr

        chroma = librosa.feature.chroma_stft(y=y[start:end], sr=sr)
        x = []
        for z in chroma:
            x.append(sum(z))

        R_max = -1
        for j in range(24):
            R = scipy.stats.pearsonr(x, KS_key[j])[0]
            if R > R_max:
                R_max = R
                music_key = j

        pieces_gen += 1
        if music_key == label_bps[i]:
            correct_gen += 1
            
        ref_key = list_key[music_key]
        est_key = list_key[label_bps[i]]
        correct_new_gen += mir_eval.key.weighted_score(ref_key, est_key)
            
    pieces_all += pieces_gen
    correct_all += correct_gen
    correct_new_all += correct_new_gen
    
    print('{:<20}'.format(file + ' ACC: ') + str(correct_gen/pieces_gen))
    print('{:<20}'.format(file + ' new ACC: ') + str(correct_new_gen/pieces_gen))

1.wav ACC:          0.38721804511278196
1.wav new ACC:      0.4107769423558898
11.wav ACC:         0.5909943714821764
11.wav new ACC:     0.6265478424014984
12.wav ACC:         0.35365853658536583
12.wav new ACC:     0.3890243902439026
13.wav ACC:         0.37948717948717947
13.wav new ACC:     0.38358974358974357
14.wav ACC:         0.34782608695652173
14.wav new ACC:     0.4184782608695657
16.wav ACC:         0.2727272727272727
16.wav new ACC:     0.2922899884925202
18.wav ACC:         0.6307541625857003
18.wav new ACC:     0.6938295788442705
19.wav ACC:         0.7333333333333333
19.wav new ACC:     0.7656140350877191
20.wav ACC:         0.5266955266955267
20.wav new ACC:     0.528138528138528
21.wav ACC:         0.3119741100323625
21.wav new ACC:     0.36336569579288064
22.wav ACC:         0.39045553145336226
22.wav new ACC:     0.43731019522776643
23.wav ACC:         0.2498410680228862
23.wav new ACC:     0.2867768595041325
24.wav ACC:         0.5116851168511685
24.wav new ACC:   

In [47]:
print('BPS-FH overall ACC: ' + str(correct_all/pieces_all))
print('BPS-FH overall new ACC: ' + str(correct_new_all/pieces_all))

BPS-FH overall ACC: 0.43083084097936836
BPS-FH overall new ACC: 0.46435342424088816


## A-MAPS

In [106]:
pieces_all = 0
correct_all = 0
correct_new_all = 0

for file in os.listdir('A-MAPS/'):
    
    midi_data = pretty_midi.PrettyMIDI('A-MAPS/' + file)
    chroma = pretty_midi.Instrument.get_chroma(midi_data)
    key_time = midi_data.key_signature_changes
    end_time = midi_data.get_end_time()

    label_amaps = []
    key_num = -1
    start = 0

    for key in key_time:
        for i in range(math.floor(key.time)-start):
            label_amaps.append(key_num)
        start = math.floor(key.time)
        if key.key_number < 12:
            key_num = (key.key_number+3)%12
        else:
            key_num = (key.key_number+3)%12 + 12

    for i in range(math.floor(end_time)-start):
        label_amaps.append(key_num)

    pieces_gen = 0
    correct_gen = 0
    correct_new_gen = 0

    for i in range(math.floor(end_time)):    
        if i < 15:
            start = 0
            end = (i+15) * 100
        elif i > math.floor(end_time)-15:
            start = (i-15) * 100
            end = math.floor(end_time) * 100
        else:
            start = (i-15) * 100
            end = (i+15) * 100

        x = []
        for z in chroma:
            x.append(sum(z[start:end]))

        R_max = -1
        for j in range(24):
            R = scipy.stats.pearsonr(x, KS_key[j])[0]
            if R > R_max:
                R_max = R
                music_key = j

        pieces_gen += 1
        if music_key == label_amaps[i]:
            correct_gen += 1

        ref_key = list_key[music_key]
        est_key = list_key[label_amaps[i]]
        correct_new_gen += mir_eval.key.weighted_score(ref_key, est_key)

    pieces_all += pieces_gen
    correct_all += correct_gen
    correct_new_all += correct_new_gen

In [107]:
print('A-MAPS overall ACC: ' + str(correct_all/pieces_all))
print('A-MAPS overall new ACC: ' + str(correct_new_all/pieces_all))

A-MAPS overall ACC: 0.35838773151763126
A-MAPS overall new ACC: 0.4331136583689918
